In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as mplot
import matplotlib.pyplot as plt 
import matplotlib.patches as patches

from matplotlib.colors import ListedColormap
import os
import subprocess
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score,  precision_recall_fscore_support
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler 
 
from sklearn.tree import export_graphviz
import pydotplus
import seaborn as sns

import openpyxl
import random
  

import tensorflow as tf
import keras
from keras import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.datasets import mnist 
 
from tensorflow.keras import layers, Input
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.metrics import AUC, Precision, Recall, F1Score
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

import pickle 
import shap 
from tabulate import tabulate 

# Define custom metrics
recall = Recall()
precision = Precision()
f1_score = F1Score()
auc = AUC()



In [ ]:
dataSetIndex = 6
needToMakePictureOfTrees = 0
''' ---------------------------------------------------------- '''
dataSetFilePath = ""
dataSetName = ""
dataSetResultDirectory = "./"

if(dataSetIndex == 0):
    dataSetFilePath = "./heartDisease/0_statLog_dataSet.csv"
    dataSetName = "SateLog_DataSet"
elif (dataSetIndex == 1):
    dataSetFilePath = "./heartDisease/1_heart_statlog_cleveland_hungary_final.csv"
    dataSetName = "ALL_StateLog_CleveLand_Hungary"
elif (dataSetIndex == 2):
    dataSetFilePath = "./heartDisease/2_cleveland.csv"
    dataSetName = "Cleveland"
elif (dataSetIndex == 3):
    dataSetFilePath = "./heartDisease/3_framingham.csv"
    dataSetName = "framingham"
elif (dataSetIndex == 4):
    dataSetFilePath = "./heartDisease/4_CardiacPrediction.xlsx"
    dataSetName = "CardiacPrediction"
elif (dataSetIndex == 5):
    dataSetFilePath = "./heartDisease/5_CardiacPredictionLessDimensions.xlsx"
    dataSetName = "CardiacPrediction"
elif (dataSetIndex == 6):
    dataSetFilePath = "./heartDisease/6_CardiacPredictionFewDimensions.xlsx"
    dataSetName = "CardiacPrediction"
else:
    dataSetFilePath = ""
    dataSetName = ""

if(dataSetIndex==4 or dataSetIndex==5 or dataSetIndex==6):
    #fileData = pd.read_excel(dataSetFilePath, sheet_name='CoroHeartDis')
    fileData = pd.read_excel(dataSetFilePath)
else:
    fileData = pd.read_csv(dataSetFilePath)

print("Shape of fileData: {}".format(fileData.shape))
print("Column Headings: {}".format(fileData.__dataframe__().column_names()))
print("Number of Records: {}".format(fileData.__dataframe__().num_rows()))


missingValues = fileData.isnull().any().sum()
print(f"\nNumber of Missing Values: {missingValues}")

num_rows_before = fileData.shape[0]
# Remove duplicate records based on all columns
fileData.drop_duplicates(inplace=True)
# Check the number of rows after removing duplicates
num_rows_after = fileData.shape[0]
# Print the number of duplicate records removed
num_duplicates_removed = num_rows_before - num_rows_after
print(f"Number of duplicate records removed: {num_duplicates_removed}")
 
 # Preprocess Steps from the ChatGPT
# 1. Handling Missing Values:
fileData = fileData.dropna()
print("Shape of fileData: {}".format(fileData.shape))              
#fileData.replace({'?': np.nan}).dropna().astype(float)
#fileData = fileData.fillna(0) 

fileData = fileData.fillna(0) 

print("Shape of fileData End: {}".format(fileData.shape))

finalResultTable = [ ['Index', 'Method', 'Accuracy %','Recall %','Precision %','F1 Score','AUC'], ]  


X = fileData.drop(fileData.__dataframe__().column_names()[-1], axis=1)  # Features
X = X.drop("Gender", axis=1)  
#cols = ['Gender', 'Age','Annual-Family-Income', 'Cholesterol', 'Diabetes', 'Triglycerides', 'Red-Cell-Distribution-Width', 'X60-sec-pulse', 'Height', 'Albumin', 'Blood-Rel-Stroke', 'Blood-Rel-Diabetes', 'HDL', 'Moderate-work','Iron', 'Hemoglobin','Protein', 'SEQN'   ] 
#cols = ['Age','Gender','Blood-Rel-Stroke','Triglycerides','Blood-Rel-Diabetes','Cholesterol','Platelet-count','Diabetes','Albumin','Hemoglobin','Moderate-work','Diastolic','Protein','Height','X60-sec-pulse','White-Blood-Cells','Bilirubin','Hematocrit','HDL','Systolic' ] 
#X = fileData[cols]

Y = fileData[fileData.__dataframe__().column_names()[-1]]  # Labels

columns = fileData.__dataframe__().column_names() 
totalRecords = (fileData.__dataframe__().num_rows())

print("\n")
print("columns of x:: {} \n\n and features of X: {}".format(len(X.columns), X.columns))

dataSetResultDirectory = "./"
dataSetResultDirectory += ("DatasetResults_MLP_imBalance_SHAP_Final")
dataSetResultDirectory += "/"
if not os.path.isdir(dataSetResultDirectory):
    os.makedirs(dataSetResultDirectory)

dataSetName += "_{}".format(fileData.shape)



In [ ]:
print("Shape of fileData: {} , target Len:{}".format(fileData.shape, len(Y)))
print("X: {} , Y:{}".format(X.shape, Y.shape))
#print("\n\nX: head:: \n{}".format(X.head()))
#print("\n\nY: head::\n {}".format(Y.head()))

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)
  
print("\n X Train: Shape:: {}".format(X_train.shape))
print(" X Test: Shape:: {}".format(X_test.shape))  
  
positiveClass =  0
negativeClass = 0
for i in y_train:
    if(i == 0):
        negativeClass += 1
    if(i == 1):
        positiveClass += 1
print("Train DataSet Positive Class Records:: {}".format(positiveClass)) 
print("Train DataSet Negative Class Records:: {}".format(negativeClass)) 
print("Train DataSet Total Records:: {}".format(positiveClass + negativeClass)) 
print("\n\n") 

positiveClass =  0
negativeClass = 0
for i in y_test:
    if(i == 0):
        negativeClass += 1
    if(i == 1):
        positiveClass += 1
print("Test DataSet Positive Class Records:: {}".format(positiveClass)) 
print("Test DataSet Negative Class Records:: {}".format(negativeClass)) 
print("Test DataSet Total Records:: {}".format(positiveClass + negativeClass)) 



In [ ]:
def makeConfusionMatrixPic(method, dataSet, classifierObj , X_test, y_test, predicted_Y):
    display = ConfusionMatrixDisplay.from_estimator(classifierObj, X_test, y_test, display_labels=['Healthy', "Heart Disease"], cmap=mplot.cm.Blues) #, normalize="true"
    display.ax_.set_title("Confusion Matrix ({} Model)".format(method))
    ax_.set_xlabel('\nPredicted Values')
    ax_.set_ylabel('Actual Values ')


    accuracyString ="Accuracy {}: {:.2f}".format(method, accuracy_score(y_test, predicted_Y)*100.0 ) 
    recallString =  'Recall {}: {:.2f}'.format(method, recall_score(y_test, predicted_Y) * 100.0)
    precisionString = 'Precision {}: {:.2f}'.format(method, precision_score(y_test, predicted_Y) * 100.0) 
    dataSetString = "Dataset: {}".format(dataSet)

    
    if(classifierObj.n_features_in_ > 10):
        featureListString = 'Total Features: {}'.format(classifierObj.n_features_in_) 
    else:
        featureListString = 'Features: {}'.format(classifierObj.feature_names_in_) 
    
    display.figure_.text(0.010, -0.05,  accuracyString, horizontalalignment='left', wrap=False )  
    display.figure_.text(0.010, -0.09,  recallString, horizontalalignment='left', wrap=False )      
    display.figure_.text(0.010, -0.13,  precisionString, horizontalalignment='left', wrap=False ) 
    display.figure_.text(0.010, -0.17,  dataSetString, horizontalalignment='left', wrap=False ) 
    display.figure_.text(0.010, -0.28,  featureListString, horizontalalignment='left', wrap=False ) 
 
    picturePath = "{}Confusion_Matrix_{}_{}.png".format(dataSetResultDirectory, method, dataSetName)
    mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
    #print("{} Confusion Matrix saved:: path: {}".format(method, picturePath))
    #os.startfile(picturePath)
    mplot.show()
    mplot.close()


In [ ]:
X_train_normalized = tf.keras.utils.normalize(X_train, axis=1)
X_test_normalized = tf.keras.utils.normalize(X_test, axis=1)

scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test) 
# Our vectorized labels

X_train_f32 = np.asarray(X_train).astype(np.float32)  #.astype('float32').reshape((-1,1))
X_test_f32 = np.asarray(X_test).astype(np.float32)

#y_train_scaler = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test_scaler = np.asarray(y_test).astype('float32').reshape((-1,1))

# Separate features and target variable
features = X_train_scaler # data.iloc[:, :-1]
target = np.asarray(y_train).astype('float64').reshape((-1,1))  #data['CoronaryHeartDisease']
 
print("X_train shape: {}   and dType: {}".format(X_train.shape, len(X_train.columns)))
print("X_train_scaler shape: {}   and dType: {}".format(X_train_scaler.shape, X_train_scaler.dtype))
print("X_test_scaler shape: {}   and dType: {}".format(X_test_scaler.shape, X_test_scaler.dtype)) 

print("y_train shape: {}   and dType: {}".format(y_train.shape, y_train.dtype))  
print("y_test_scaler shape: {}   and dType: {}".format(y_test_scaler.shape, y_test_scaler.dtype))  

print("features shape: {}   and dType: {}".format(features.shape, features.dtype))
print("target shape: {}   and dType: {}".format(target.shape, target.dtype)) 

In [ ]:
tf.keras.backend.clear_session()   
totalFeatures= features.shape[1] 
print("Total Feature for number of node in Layer: {}".format(totalFeatures)) 
input_layer = Input(shape=(totalFeatures,))
# Traditional neural network part
x = layers.Dense(int(totalFeatures), activation='relu')(input_layer)
x = layers.Dense(int(totalFeatures*0.75), activation='relu')(x)
x = layers.Dense(int(totalFeatures*0.50), activation='relu')(x)
x = layers.Dense(int(totalFeatures*0.25), activation='relu')(x) 

output_layer = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer) 
optimizer = AdamW(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', Recall(), Precision(), AUC()])  # Add custom metrics

# Display the model summary
model.summary()
 


In [ ]:
numberOfEpochs = 32
batchSizeOfTraining = 25
history = 0
history = model.fit(features, target, epochs=numberOfEpochs, batch_size=batchSizeOfTraining)

model2 = model


In [ ]:
# Plotting training accuracy
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], marker='o', linestyle='-', color='b')
plt.title('Training Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True)
picturePath = "{}3.Model_training_Accuracy_{}_epoches_{}.png".format(dataSetResultDirectory, dataSetName, numberOfEpochs)
plt.savefig(picturePath,  dpi=300, bbox_inches='tight')
plt.show()
plt.close()

method = "MLP"
# Plot Testing accuracy
plt.figure(figsize=(10, 6))
for metric_name in history.history.keys():
    plt.plot(history.history[metric_name], label=metric_name.capitalize())
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Model Training Metrics')
# Add legend
plt.legend()
picturePath = "{}Model_Training_Evaluation_{}_{}_Epoch_{}.png".format(dataSetResultDirectory, method, dataSetName, numberOfEpochs)
mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
plt.show()
plt.close()

#-------------------------------------------------------------------------------------------------------------------------------
# Evaluate the model using the evaluate method
y_test_float64 = np.asarray(y_test).astype('float64').reshape((-1,1))  #data['CoronaryHeartDisease']
loss, accuracy, recall_value, precision_value,  auc_value = model2.evaluate(X_test_scaler, y_test_float64)

# Print the results
print('Test loss: {}'.format(loss*100))
print('Test accuracy: {}'.format(accuracy*100))
print('Test recall: {}'.format(recall_value*100))
print('Test precision: {}'.format(precision_value*100))
print('Test AUC: {}'.format(auc_value*100)) 


#-------------------------------------------------------------------------------------------------------------------------------
threshold = 0.5  # Adjust the threshold as needed
binary_predictions = (model2.predict(X_test_scaler) > threshold).astype(int)
cm = confusion_matrix(y_test_float64, binary_predictions)
display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Healthy', 'Heart Disease'])
display.plot(cmap=plt.cm.Blues, values_format=".4g" ) 
display.ax_.set_title("Results {} Model".format(method),fontsize=16, fontweight='bold')
display.ax_.set_xlabel('\nPredicted Values')
display.ax_.set_ylabel('Actual Values ')

accuracyString ="Accuracy {}: {:.2f}%".format(method, accuracy*100.0 ) 
recallString =  'Recall {}: {:.2f}%'.format(method, recall_value* 100.0)
precisionString = 'Precision {}: {:.2f}%'.format(method, precision_value * 100.0) 
#f1String = "F1 Score: {:.2f}".format(f1_score_value[0] * 100.0)
featureListString = "AUC Score: {:.2f}%".format(auc_value * 100.0)
totaldataSetString = "Total Dataset records: {}".format((len(features)+len(X_test_scaler)))
testingdataSetString = "Testing records: {} , {:.1f}%".format(len(X_test_scaler), ( ( len(X_test_scaler) / (len(features)+len(X_test_scaler)) )*100.0  ))
numberOfEpochsString = "Number of Epoches: {}".format(numberOfEpochs)
batchSizeOfTrainingString = "BatchSize for Epoch: {}".format(batchSizeOfTraining)

display.figure_.text(0.010, -0.05,  accuracyString, horizontalalignment='left', wrap=False )  
display.figure_.text(0.010, -0.09,  recallString, horizontalalignment='left', wrap=False )      
display.figure_.text(0.010, -0.13,  precisionString, horizontalalignment='left', wrap=False ) 
#display.figure_.text(0.010, -0.17,  f1String, horizontalalignment='left', wrap=False ) 
display.figure_.text(0.010, -0.21,  featureListString, horizontalalignment='left', wrap=False ) 
display.figure_.text(0.010, -0.26,  totaldataSetString, horizontalalignment='left', wrap=False ) 
display.figure_.text(0.010, -0.30,  testingdataSetString, horizontalalignment='left', wrap=False ) 
display.figure_.text(0.010, -0.34,  numberOfEpochsString, horizontalalignment='left', wrap=False ) 
display.figure_.text(0.010, -0.38,  batchSizeOfTrainingString, horizontalalignment='left', wrap=False ) 
 
picturePath = "{}Model_Evaluation_{}_{}_Epoch_{}.png".format(dataSetResultDirectory, method, dataSetName, numberOfEpochs)
mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
#print("{} Confusion Matrix saved:: path: {}".format(method, picturePath))
mplot.show()
mplot.close()

#-------------------------------------------------------------------------------------------------------------------------------
# Calculate AUC
auc_score = roc_auc_score(y_test, binary_predictions)
# Calculate ROC Curve
fpr, tpr, _ = roc_curve(y_test, binary_predictions)
# Plot ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(auc_score))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')

picturePath = "{}Model_Evaluation_ROC_{}_{}_Epoch_{}.png".format(dataSetResultDirectory, method, dataSetName, numberOfEpochs)
mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
plt.show()
plt.close()



#-------------------------------------------------------------------------------------------------------------------------------
csvPath = "{}Model_training_accuracy_and_evaluations_{}_{}_Epoch_{}.xlsx".format(dataSetResultDirectory, method, dataSetName, numberOfEpochs)
# Create a new workbook
wb = openpyxl.Workbook()
ws = wb.active  # Get the active worksheet
str22 = (classification_report(y_test, binary_predictions)) 
str22.strip() 
str22.replace(" ", ",")
#print(str22)
print("-------------=-=-=-=----------------")
data_lines = str22.splitlines()  # Split by newlines
str22 = str("\n\n\n Training accuracy:\n"  )
data_lines += str22.splitlines()  # Split by newlines
str22 = str(history.history['accuracy'])
str22.strip() 
str22.replace(" ", ",")
str22.replace("[", "")
str22.replace("]", "")
#print(str22)
data_lines += str22.splitlines()  # Split by newlines


str22 = str("\n\n\n ROC Values:\n"  )
data_lines += str22.splitlines()  # Split by newlines
str22 = str("False Positive Rate:, {}".format(fpr))
data_lines += str22.splitlines()  # Split by newlines
str22 = str("True Positive Rate:, {}".format(tpr))
data_lines += str22.splitlines()  # Split by newlines
str22 = str("ROC curve, (AUC = {})".format(auc_score))
data_lines += str22.splitlines()  # Split by newlines



print(data_lines)
# Split each line into a list (comma-separated values)
xlsFileData = [line.split(",") for line in data_lines]
# Write the data to the worksheet, starting from row 1
for row_index, row in enumerate(xlsFileData):
    for col_index, value in enumerate(row):
        ws.cell(row=row_index + 1, column=col_index + 1).value = value

# Save the workbook
wb.save(csvPath)

In [ ]:
24erwer234

In [ ]:
percentage = 0.03 
featuresForShap = X_train.columns #features[0:numberOfFeatures]
#print(" Features Name: {}".format(  featuresForShap))
 
testForShap = X_test_scaler[0:500]
#print(" testForShap Name: {}".format(  testForShap))
print("TensorFlow Version: {}".format(tf.__version__))
print("SHAP Version: {}".format(shap.__version__))

shap.initjs()

In [ ]:
shap_explainer = shap.Explainer(model, feature_names=featuresForShap, masker=shap.maskers.Independent(data=testForShap)) 
shap_values = shap_explainer.shap_values(testForShap)  

# Calculate feature importance based on the mean absolute SHAP values
feature_names = X_train.columns.tolist()
# Calculate average SHAP values across all instances
avg_shap_values = np.mean(shap_values, axis=0) 
simpleExplainerShap_array = np.array(shap_values) 
mean_abs_simpleShap_values = np.mean(np.abs(simpleExplainerShap_array), axis=(0)) 
sorted_indices = np.argsort(-mean_abs_simpleShap_values) #[::-1] 
sorted_feature_names = np.array(X.columns.to_list())[sorted_indices]
sorted_shap_values = simpleExplainerShap_array[:, sorted_indices].T
#print(sorted_shap_values) 
deepShapValuesPlot = mean_abs_simpleShap_values[sorted_indices]
print("Feature Ranking:")
index = 1
for feature, importance in zip(sorted_feature_names, mean_abs_simpleShap_values[sorted_indices]): 
  print("{} , {}, {}".format(index, feature, importance))
  index = index+1
#------------------------------------------------------------------------------------------- 
howManyFeatures = 37
# Calculate feature importance based on the mean absolute SHAP values
feature_importance =  np.abs(shap_values).mean(axis=0)
top_features_indices = np.argsort(feature_importance)[::-1][:howManyFeatures]
# Select only the top features and corresponding SHAP values
print(top_features_indices)
featureNamesSHAP = X.columns[top_features_indices]
top_features = testForShap[:, top_features_indices]
top_shap_values = shap_values[:, top_features_indices]
print("\n\n--------------------------------------------------") 
print("Top SHAP Explainer values:")
for i in range(len(top_features_indices)):
    feature_index = top_features_indices[i]
    feature_name = feature_names[feature_index]
    shap_value = np.mean(np.abs(top_shap_values[:, i])) 
    #print(f"{feature_name}, {shap_value}")
#---------------------------------------------------------------------------------
str22 = 0
csvPath = "{}Model_SHAP_simpleExplainer_FeatureRanking_{}_{}_Epoch_{}.xlsx".format(dataSetResultDirectory, method, dataSetName, numberOfEpochs)
str22 = str("SimpleExplainer Top Feature List\n")
indexx = 1
for feature, importance in zip(sorted_feature_names, mean_abs_simpleShap_values[sorted_indices]): 
        str22 += str("{}, {}, {} \n".format(indexx, feature,importance)) 
        indexx += 1
wb = openpyxl.Workbook()
ws = wb.active  # Get the active worksheet
data_lines = str22.splitlines()  # Split by newlines
xlsFileData = [line.split(",") for line in data_lines] 
for row_index, row in enumerate(xlsFileData):
    for col_index, value in enumerate(row):
        ws.cell(row=row_index + 1, column=col_index + 1).value = value
wb.save(csvPath)


#---------------------------------------------------------------------------------
howManyFeatures = 10
# Calculate feature importance based on the mean absolute SHAP values
feature_importance =  np.abs(shap_values).mean(axis=0)
top_features_indices = np.argsort(feature_importance)[::-1][:howManyFeatures]
# Select only the top features and corresponding SHAP values
print(top_features_indices)
featureNamesSHAP = X.columns[top_features_indices]
top_features = testForShap[:, top_features_indices]
top_shap_values = shap_values[:, top_features_indices]
# Plot the summary plot for the top 15 features
shap.summary_plot(top_shap_values, top_features, feature_names=featureNamesSHAP, plot_type="bar", show=False)
ax = mplot.gca() 
ax.set_title("XAI SHAP Explainer Sorted  ({} Features)".format(howManyFeatures) ,fontsize=16, fontweight='bold')     
dataSetString = "Dataset:  {}".format(dataSetName)
testingDatasetString ="length of SHAP dataset: {}".format(len(testForShap))
shapTypeString ="SHAP Type: {}".format(repr(shap_explainer)) 
ax.figure.text(0.020, -0.05,  dataSetString, horizontalalignment='left', wrap=False )  
ax.figure.text(0.020, -0.09,  testingDatasetString, horizontalalignment='left', wrap=False )   
ax.figure.text(0.020, -0.13,  shapTypeString, horizontalalignment='left', wrap=False )   
picturePath = "{}XAI_SHAP_Explainer_Bar_{}_{}_Sorted_numberOfSamples_{}_Features_{}.png".format(dataSetResultDirectory, method, dataSetName, len(testForShap),howManyFeatures)
mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
#mplot.show()
#os.startfile(picturePath)
mplot.close()
# Plot the summary plot for the top 15 features
shap.summary_plot(top_shap_values, top_features, feature_names=featureNamesSHAP, show=False)
ax = mplot.gca() 
ax.set_title("XAI SHAP Explainer Sorted  ({} Features)".format(howManyFeatures) ,fontsize=16, fontweight='bold')     
dataSetString = "Dataset:  {}".format(dataSetName)
testingDatasetString ="length of SHAP dataset: {}".format(len(testForShap))
shapTypeString ="SHAP Type: {}".format(repr(shap_explainer)) 
ax.figure.text(0.020, -0.05,  dataSetString, horizontalalignment='left', wrap=False )  
ax.figure.text(0.020, -0.09,  testingDatasetString, horizontalalignment='left', wrap=False )   
ax.figure.text(0.020, -0.13,  shapTypeString, horizontalalignment='left', wrap=False )   
picturePath = "{}XAI_SHAP_Explainer_SummaryPlot_{}_{}_Sorted_numberOfSamples_{}_Features_{}.png".format(dataSetResultDirectory, method, dataSetName, len(testForShap),howManyFeatures)
mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
#mplot.show()
#os.startfile(picturePath)
mplot.close()

howManyFeatures = 15
# Calculate feature importance based on the mean absolute SHAP values
feature_importance =  np.abs(shap_values).mean(axis=0)
top_features_indices = np.argsort(feature_importance)[::-1][:howManyFeatures]
# Select only the top features and corresponding SHAP values
print(top_features_indices)
featureNamesSHAP = X.columns[top_features_indices]
top_features = testForShap[:, top_features_indices]
top_shap_values = shap_values[:, top_features_indices]
# Plot the summary plot for the top 15 features
shap.summary_plot(top_shap_values, top_features, feature_names=featureNamesSHAP, plot_type="bar", show=False)
ax = mplot.gca() 
ax.set_title("XAI SHAP Explainer Sorted  ({} Features)".format(howManyFeatures) ,fontsize=16, fontweight='bold')     
dataSetString = "Dataset:  {}".format(dataSetName)
testingDatasetString ="length of SHAP dataset: {}".format(len(testForShap))
shapTypeString ="SHAP Type: {}".format(repr(shap_explainer)) 
ax.figure.text(0.020, -0.05,  dataSetString, horizontalalignment='left', wrap=False )  
ax.figure.text(0.020, -0.09,  testingDatasetString, horizontalalignment='left', wrap=False )   
ax.figure.text(0.020, -0.13,  shapTypeString, horizontalalignment='left', wrap=False )   
picturePath = "{}XAI_SHAP_Explainer_Bar_{}_{}_Sorted_numberOfSamples_{}_Features_{}.png".format(dataSetResultDirectory, method, dataSetName, len(testForShap),howManyFeatures)
mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
#mplot.show()
#os.startfile(picturePath)
mplot.close()
# Plot the summary plot for the top 15 features
shap.summary_plot(top_shap_values, top_features, feature_names=featureNamesSHAP, show=False)
ax = mplot.gca() 
ax.set_title("XAI SHAP Explainer Sorted  ({} Features)".format(howManyFeatures) ,fontsize=16, fontweight='bold')     
dataSetString = "Dataset:  {}".format(dataSetName)
testingDatasetString ="length of SHAP dataset: {}".format(len(testForShap))
shapTypeString ="SHAP Type: {}".format(repr(shap_explainer)) 
ax.figure.text(0.020, -0.05,  dataSetString, horizontalalignment='left', wrap=False )  
ax.figure.text(0.020, -0.09,  testingDatasetString, horizontalalignment='left', wrap=False )   
ax.figure.text(0.020, -0.13,  shapTypeString, horizontalalignment='left', wrap=False )   
picturePath = "{}XAI_SHAP_Explainer_SummaryPlot_{}_{}_Sorted_numberOfSamples_{}_Features_{}.png".format(dataSetResultDirectory, method, dataSetName, len(testForShap),howManyFeatures)
mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
#mplot.show()
#os.startfile(picturePath)
mplot.close()



howManyFeatures = 20
# Calculate feature importance based on the mean absolute SHAP values
feature_importance =  np.abs(shap_values).mean(axis=0)
top_features_indices = np.argsort(feature_importance)[::-1][:howManyFeatures]
# Select only the top features and corresponding SHAP values
print(top_features_indices)
featureNamesSHAP = X.columns[top_features_indices]
top_features = testForShap[:, top_features_indices]
top_shap_values = shap_values[:, top_features_indices]
# Plot the summary plot for the top 15 features
shap.summary_plot(top_shap_values, top_features, feature_names=featureNamesSHAP, plot_type="bar", show=False)
ax = mplot.gca() 
ax.set_title("XAI SHAP Explainer Sorted  ({} Features)".format(howManyFeatures) ,fontsize=16, fontweight='bold')     
dataSetString = "Dataset:  {}".format(dataSetName)
testingDatasetString ="length of SHAP dataset: {}".format(len(testForShap))
shapTypeString ="SHAP Type: {}".format(repr(shap_explainer)) 
ax.figure.text(0.020, -0.05,  dataSetString, horizontalalignment='left', wrap=False )  
ax.figure.text(0.020, -0.09,  testingDatasetString, horizontalalignment='left', wrap=False )   
ax.figure.text(0.020, -0.13,  shapTypeString, horizontalalignment='left', wrap=False )   
picturePath = "{}XAI_SHAP_Explainer_Bar_{}_{}_Sorted_numberOfSamples_{}_Features_{}.png".format(dataSetResultDirectory, method, dataSetName, len(testForShap),howManyFeatures)
mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
#mplot.show()
#os.startfile(picturePath)
mplot.close()
# Plot the summary plot for the top 15 features
shap.summary_plot(top_shap_values, top_features, feature_names=featureNamesSHAP, show=False)
ax = mplot.gca() 
ax.set_title("XAI SHAP Explainer Sorted  ({} Features)".format(howManyFeatures) ,fontsize=16, fontweight='bold')     
dataSetString = "Dataset:  {}".format(dataSetName)
testingDatasetString ="length of SHAP dataset: {}".format(len(testForShap))
shapTypeString ="SHAP Type: {}".format(repr(shap_explainer)) 
ax.figure.text(0.020, -0.05,  dataSetString, horizontalalignment='left', wrap=False )  
ax.figure.text(0.020, -0.09,  testingDatasetString, horizontalalignment='left', wrap=False )   
ax.figure.text(0.020, -0.13,  shapTypeString, horizontalalignment='left', wrap=False )   
picturePath = "{}XAI_SHAP_Explainer_SummaryPlot_{}_{}_Sorted_numberOfSamples_{}_Features_{}.png".format(dataSetResultDirectory, method, dataSetName, len(testForShap),howManyFeatures)
mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
#mplot.show()
#os.startfile(picturePath)
mplot.close()




In [ ]:
deepExplainer = shap.Explainer(model, features[0:int(1000)], feature_names=featuresForShap)
#deepExplainer = shap.Explainer(model)
deepTestValues =  testForShap[0:int(400)] 
shap_FitData = deepExplainer(deepTestValues)

print(shap_FitData.shape) #output: (4177, 8)
howManyWaterFallPlotsRequired = int(len(shap_FitData)//2.0)
print("\nHow many Waterfall plots are generated: {}".format(howManyWaterFallPlotsRequired))

# Water fall Plots
displayNumberOfFeatures = 20
shapDirectory = ""
shapDirectory += dataSetResultDirectory + ("SHAP_WaterFall_Epoches_{}_Features_{}".format(numberOfEpochs, displayNumberOfFeatures))
shapDirectory += "/"
if not os.path.isdir(shapDirectory):
    os.makedirs(shapDirectory)
index = 1
for sample in shap_FitData[0:howManyWaterFallPlotsRequired]:
    plt.figure(figsize=(8, 16))
    plt.switch_backend('Agg')
    shap.waterfall_plot(sample, max_display=displayNumberOfFeatures)
    ax = mplot.gca() 
    ax.set_title("XAI SHAP Waterfall Plot of Sample {}".format(index) ,fontsize=16, fontweight='bold')     
    dataSetString = "Dataset:  {}".format(dataSetName)
    testingDatasetString ="length of SHAP dataset: {}".format(len(testForShap))
    ax.figure.text(0.020, -0.05,  dataSetString, horizontalalignment='left', wrap=False )  
    ax.figure.text(0.020, -0.09,  testingDatasetString, horizontalalignment='left', wrap=False )   
    picturePath = "{}XAI_SHAP_Explainer_WaterFall_{}_TotalSHAPvalues_{}_Epoch_{}.png".format(shapDirectory, index,  len(testForShap), numberOfEpochs) 
    plt.savefig(picturePath,  dpi=300, bbox_inches='tight')
    #mplot.show()
    #os.startfile(picturePath)
    plt.close()
    plt.clf()
    index = index+1


displayNumberOfFeatures = 15
shapDirectory = ""
shapDirectory += dataSetResultDirectory + ("SHAP_WaterFall_Epoches_{}_Features_{}".format(numberOfEpochs, displayNumberOfFeatures))
shapDirectory += "/"
if not os.path.isdir(shapDirectory):
    os.makedirs(shapDirectory)
index = 1
for sample in shap_FitData[0:howManyWaterFallPlotsRequired]:
    plt.figure(figsize=(8, 6))
    plt.switch_backend('Agg')
    shap.waterfall_plot(sample, max_display=displayNumberOfFeatures)
    ax = mplot.gca() 
    ax.set_title("XAI SHAP Waterfall Plot of Sample {}".format(index) ,fontsize=16, fontweight='bold')     
    dataSetString = "Dataset:  {}".format(dataSetName)
    testingDatasetString ="length of SHAP dataset: {}".format(len(testForShap))
    ax.figure.text(0.020, -0.05,  dataSetString, horizontalalignment='left', wrap=False )  
    ax.figure.text(0.020, -0.09,  testingDatasetString, horizontalalignment='left', wrap=False )   
    picturePath = "{}XAI_SHAP_Explainer_WaterFall_{}_TotalSHAPvalues_{}_Epoch_{}.png".format(shapDirectory, index,  len(testForShap), numberOfEpochs) 
    plt.savefig(picturePath,  dpi=300, bbox_inches='tight')
    #mplot.show()
    #os.startfile(picturePath)
    plt.close()
    plt.clf()
    index = index+1


displayNumberOfFeatures = 10
shapDirectory = ""
shapDirectory += dataSetResultDirectory + ("SHAP_WaterFall_Epoches_{}_Features_{}".format(numberOfEpochs, displayNumberOfFeatures))
shapDirectory += "/"
if not os.path.isdir(shapDirectory):
    os.makedirs(shapDirectory)
index = 1
for sample in shap_FitData[0:howManyWaterFallPlotsRequired]:
    plt.figure(figsize=(8, 6))
    plt.switch_backend('Agg')
    shap.waterfall_plot(sample, max_display=displayNumberOfFeatures)
    ax = mplot.gca() 
    ax.set_title("XAI SHAP Waterfall Plot of Sample {}".format(index) ,fontsize=16, fontweight='bold')     
    dataSetString = "Dataset:  {}".format(dataSetName)
    testingDatasetString ="length of SHAP dataset: {}".format(len(testForShap))
    ax.figure.text(0.020, -0.05,  dataSetString, horizontalalignment='left', wrap=False )  
    ax.figure.text(0.020, -0.09,  testingDatasetString, horizontalalignment='left', wrap=False )   
    picturePath = "{}XAI_SHAP_Explainer_WaterFall_{}_TotalSHAPvalues_{}_Epoch_{}.png".format(shapDirectory, index,  len(testForShap), numberOfEpochs) 
    plt.savefig(picturePath,  dpi=300, bbox_inches='tight')
    #mplot.show()
    #os.startfile(picturePath)
    plt.close()
    plt.clf()
    index = index+1
